In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv2D, Flatten, MaxPooling2D, AveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
DATA_DIR='/kaggle/input/plant-pathology-2021-fgvc8/train_images/'
df=pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')
LR = 1e-3
N_EPOCHS = 200
BATCH_SIZE = 512
DROPOUT = 0.2
IMG_ROWS, IMG_COLS = 128, 128
SEED=42

In [ ]:
df.labels.value_counts()

In [ ]:
img=cv2.imread(DATA_DIR+'803270d4d78faeba.jpg')
img2=cv2.resize(img,(IMG_ROWS, IMG_COLS))
print(img.shape)
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.axis('off')
ax2.axis('off')
ax1.imshow(img)
ax2.imshow(img2)

In [ ]:
def train_data_import(df):
    x_train,y_train=[],[]
    
    y=np.array(df.labels)
    
    count=0
    for path in list(df.image):
        count+=1
        if count%100==0:
            print(count)
        x_train.append(cv2.resize(cv2.imread(DATA_DIR+path),(IMG_ROWS, IMG_COLS)))
    
    le=LabelEncoder()
    le.fit(df.labels.value_counts().index)
    y_train=le.transform(y)
    
    x_train=np.array(x_train).astype('float32')
#     x_train = x_train.reshape(x_train.shape[0], IMG_ROWS, IMG_COLS, 1)
    x_train/=255
    

    return x_train,y_train

TEST_DIR='/kaggle/input/plant-pathology-2021-fgvc8/test_images/'
def test_data_index_import(TEST_DIR):
    x_test=[]
    testindex=[]
    for path in os.listdir(TEST_DIR):
        x_test.append(cv2.resize(cv2.imread(TEST_DIR+path),(IMG_ROWS, IMG_COLS)))
        testindex.append(path)
    
    x_test=np.array(x_test).astype('float32')
#     x_test = x_test.reshape(x_test.shape[0], IMG_ROWS, IMG_COLS, 1)
    x_test/=255
    
    return x_test,testindex

In [ ]:
xtrain,ytrain=train_data_import(df)

In [ ]:
xtest,test_index=test_data_index_import(TEST_DIR)
x_train,x_val,y_train,y_val=train_test_split(xtrain, ytrain, random_state=SEED, test_size=0.2,shuffle=True,stratify=ytrain)
y_train, y_val = to_categorical(y_train, num_classes=12), to_categorical(y_val, num_classes=12)

In [ ]:
def get_model():
    

    model = Sequential([
    Conv2D(16, (3,3),   input_shape = (IMG_ROWS, IMG_COLS, 3),activation="relu"),
    
    BatchNormalization(),
        
    MaxPooling2D((2,2)),
    Conv2D(32, (3,3), activation="relu"),
        
    BatchNormalization(),
        
    AveragePooling2D((2,2)),
    Flatten(),
        
    Dense(400, activation="tanh"),
        
    Dropout(DROPOUT),
        
    BatchNormalization(),
        
    Dense(12, activation="softmax")
    ])

    return model

   


In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, mode='auto'
                               , restore_best_weights=True)

model=get_model()

model.compile(optimizer=Adam(lr=LR), loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS, validation_data=(x_val, y_val),callbacks=[early_stopping])

In [ ]:
print("F1 score", f1_score(np.argmax(model.predict(x_val),axis=1),np.argmax(y_val,axis=1), average = 'macro'))

In [ ]:
submission=pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
def label(lb_index):
    lblst=[]
    for lb in lb_index:
        lblst.append(list(df.labels.value_counts().index)[lb])
    return lblst


In [ ]:
# test
submission=pd.DataFrame({'image':test_index,'labels':label(np.argmax(model.predict(xtest),axis=1))})

In [ ]:
submission.to_csv('submission.csv',index=False)